<a href="https://colab.research.google.com/github/Phund4Bot/DZ/blob/main/kurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h>Spark ML <h>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f7f2b352b4c6f67ce40c90a407fc1a4ead54e802d3e23c08364bcfd298860bf6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [197]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer # Выполнение энкодинга
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import split
from pyspark.sql.functions import col
from pyspark.sql.functions import when





In [3]:
spark = SparkSession.builder.appName("PySparkParkinsonJob").getOrCreate()

#Создаём Spark сессию

In [4]:
spark

In [186]:
winequality_red_df = spark.read.csv('winequality-red.csv', header=True, inferSchema=True)


# Разделение колонки на отдельные столбцы
split_cols = winequality_red_df['"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"']

split_cols = split(split_cols, ';')

# Добавление новых столбцов к DataFrame
winequality_red_df = winequality_red_df.withColumn('fixed_acidity', split_cols.getItem(0))
winequality_red_df = winequality_red_df.withColumn('volatile_acidity', split_cols.getItem(1))
winequality_red_df = winequality_red_df.withColumn('citric_acid', split_cols.getItem(2))
winequality_red_df = winequality_red_df.withColumn('residual_sugar', split_cols.getItem(3))
winequality_red_df = winequality_red_df.withColumn('chlorides', split_cols.getItem(4))
winequality_red_df = winequality_red_df.withColumn('free_sulfur_dioxide', split_cols.getItem(5))
winequality_red_df = winequality_red_df.withColumn('total_sulfur_dioxide', split_cols.getItem(6))
winequality_red_df = winequality_red_df.withColumn('density', split_cols.getItem(7))
winequality_red_df = winequality_red_df.withColumn('pH', split_cols.getItem(8))
winequality_red_df = winequality_red_df.withColumn('sulphates', split_cols.getItem(9))
winequality_red_df = winequality_red_df.withColumn('alcohol', split_cols.getItem(10))
winequality_red_df = winequality_red_df.withColumn('quality', split_cols.getItem(11))

# Удаление исходной колонки
winequality_red_df = winequality_red_df.drop('"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"')

for column_name in winequality_red_df.columns:
    winequality_red_df = winequality_red_df.withColumn(column_name, col(column_name).cast('double'))

In [187]:
winequality_red_df.show(20)



+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    5.0|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [37]:
# Поработаем с категориальными признаками.

#Функция StringIndexer индексирует (энкодит) строки
#Концепция, лежащая в основе индексирования строк, очень интуитивно понятна.
#Мы просто заменяем каждую категорию номером.

# sex_index = StringIndexer(inputCol="Genre", outputCol="Genre_index")  #StringIndexer - это Estimator, который нам формирует Transformer для преобразования данных
# embarked_index = StringIndexer(inputCol='Embarked', outputCol="Embarked_index")

In [38]:
# теперь применим новые столбцы к нашему датафрейму. Сначала обращаемся ним и выполняем операцию fit на нашем датафрейме.
#После чего мы получим функцию трансформер и сможем вызвать ф-ию transform для того, чтобы выполнить преобразование наших данных

# mall_customers_df = sex_index.fit(mall_customers_df).transform(mall_customers_df)

In [202]:
# Сделайте преобразование для embarked_index
# titanic_df = embarked_index.fit(titanic_df).transform(titanic_df)
winequality_red_df = winequality_red_df.withColumn("target", when(winequality_red_df["quality"] >= 6, 1).otherwise(0))

In [203]:
winequality_red_df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|target|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|     0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|     0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    5.0|     0|
|         11.2|            0.28|       0.56|           1.9

In [204]:
# Наш dataframe готов к тому, чтобы применить к нему модель. Но мы хотим сначала выполнить векторизацию, чтобы данные можно было применять в моделе.
# Формируем список признаков, которые мы будем использовать, как фичи.
features = [col for col in winequality_red_df.columns if col != 'quality' and col != "target"]


In [205]:
feature = VectorAssembler(inputCols=features, outputCol="features")
#VectorAssembler объединяет заданный список столбцов в один векторный столбец.
# Является, пожалуй, самым важным векторным преобразователем в PySpark,
# поскольку модели машинного обучения требуют на вход векторы.

# VectorAssembler - трансформер, соответственно, вызываем метод transform  и применяем его к нашему датафрейму.
feature_vector= feature.transform(winequality_red_df)

In [262]:
feature_vector.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+--------------------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|target|            features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+--------------------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|     0|[7.4,0.7,0.0,1.9,...|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|     0|[7.8,0.88,0.0,2.6...|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|

In [207]:
(training_data, test_data) = feature_vector.randomSplit([0.8, 0.2],seed = 42)
#Формируем тренировочный и тестовый датасет. 80% данных в тренировочный, остальные - в тестовый. Seed - параметр случайности

In [182]:
training_data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|            features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          3.8|            0.31|       0.02|          11.1|    0.036|               20.0|               114.0|0.99248|3.75|     0.44|   12.4|    6.0|[3.8,0.31,0.02,11...|
|          3.9|           0.225|        0.4|           4.2|     0.03|               29.0|               118.0|  0.989|3.57|     0.36|   12.8|    8.0|[3.9,0.225,0.4,4....|
|          4.2|           0.215|       0.23|           5.1|    0.041|               64.0|               157.0|0.99688|3.42|     0.44|    8.0|    

# ML models

# LogisticRegression

In [208]:
evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")

#Сначала создаем объект evaluator. Указываем целевую колонку обучения модели - labelCol. далее указываем
# имя колонки, где будет лежать предсказанное значение  - predictionCol. И указываем метрику для оценки качества модели.

In [209]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="target", featuresCol="features") # labelCol="Survived" - целевая фича,  featuresCol="features" - фичи, которые
#используются для предсказания занчения целевой колонки

lrModel = lr.fit(training_data) # получаем обученную модель.

lr_prediction = lrModel.transform(test_data) # применяем модель на данных, получаем предсказание
lr_prediction.select("prediction", "target", "features").show(30)

+----------+------+--------------------+
|prediction|target|            features|
+----------+------+--------------------+
|       1.0|     1|[4.9,0.42,0.0,2.1...|
|       1.0|     1|[5.0,0.74,0.0,1.2...|
|       0.0|     0|[5.0,1.04,0.24,1....|
|       0.0|     0|[5.2,0.32,0.25,1....|
|       1.0|     1|[5.3,0.47,0.11,2....|
|       1.0|     1|[5.4,0.42,0.27,2....|
|       0.0|     0|[5.6,0.31,0.37,1....|
|       1.0|     0|[5.6,0.605,0.05,2...|
|       1.0|     1|[5.8,0.29,0.26,1....|
|       1.0|     1|[5.8,0.61,0.11,1....|
|       0.0|     0|[5.8,0.68,0.02,1....|
|       0.0|     0|[5.9,0.19,0.21,1....|
|       1.0|     1|[5.9,0.395,0.13,2...|
|       1.0|     1|[5.9,0.61,0.08,2....|
|       0.0|     0|[6.0,0.5,0.0,1.4,...|
|       1.0|     1|[6.0,0.58,0.2,2.4...|
|       1.0|     1|[6.1,0.4,0.16,1.8...|
|       0.0|     0|[6.2,0.58,0.0,1.6...|
|       1.0|     1|[6.3,0.51,0.13,2....|
|       1.0|     0|[6.3,0.6,0.1,1.6,...|
|       1.0|     1|[6.3,0.68,0.01,3....|
|       1.0|    

In [210]:
# Применяем оценщик (evaluator), чтобы узнать точность модели.
lr_accuracy = evaluator.evaluate(lr_prediction)
print("LogisticRegression [Accuracy] = %g"% (lr_accuracy))
print("LogisticRegression [Error] = %g " % (1.0 - lr_accuracy))

LogisticRegression [Accuracy] = 0.774545
LogisticRegression [Error] = 0.225455 


# DecisionTreeClassifier

In [212]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="target", featuresCol="features")
dt_model = dt.fit(training_data)
dt_prediction = dt_model.transform(test_data)

dt_prediction.select("prediction", "target", "features").show(5)

+----------+------+--------------------+
|prediction|target|            features|
+----------+------+--------------------+
|       1.0|     1|[4.9,0.42,0.0,2.1...|
|       1.0|     1|[5.0,0.74,0.0,1.2...|
|       1.0|     0|[5.0,1.04,0.24,1....|
|       0.0|     0|[5.2,0.32,0.25,1....|
|       1.0|     1|[5.3,0.47,0.11,2....|
+----------+------+--------------------+
only showing top 5 rows



In [213]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("DecisionTreeClassifier [Accuracy] = %g"% (dt_accuracy))
print("DecisionTreeClassifier [Error] = %g " % (1.0 - dt_accuracy))

DecisionTreeClassifier [Accuracy] = 0.72
DecisionTreeClassifier [Error] = 0.28 


# RandomForestClassifier

Реализуйте самостоятельно и оцените точность, как в примерах выше

In [216]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="target", featuresCol="features")
rf_model = rf.fit(training_data)
rf_prediction = rf_model.transform(test_data)

rf_prediction.select("prediction", "target", "features").show(5)

+----------+------+--------------------+
|prediction|target|            features|
+----------+------+--------------------+
|       1.0|     1|[4.9,0.42,0.0,2.1...|
|       1.0|     1|[5.0,0.74,0.0,1.2...|
|       1.0|     0|[5.0,1.04,0.24,1....|
|       0.0|     0|[5.2,0.32,0.25,1....|
|       1.0|     1|[5.3,0.47,0.11,2....|
+----------+------+--------------------+
only showing top 5 rows



In [217]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("DecisionTreeClassifier [Accuracy] = %g"% (rf_accuracy))
print("DecisionTreeClassifier [Error] = %g " % (1.0 - rf_accuracy))

DecisionTreeClassifier [Accuracy] = 0.792727
DecisionTreeClassifier [Error] = 0.207273 


# Gradient-boosted tree classifier

Реализуйте самостоятельно и оцените точность, как в примерах выше

# Save & Load Model

In [ ]:
# обученную модель мы можем сохранять. Сохраним модель RandomForest.
rf_model.write().overwrite().save('rf_model')

In [ ]:
# Теперь модель загрузим. Для этого сначала модключим нужный класс
from pyspark.ml.classification import RandomForestClassificationModel
type(RandomForestClassificationModel.load('rf_model'))

pyspark.ml.classification.RandomForestClassificationModel

# Pipeline

In [285]:
from pyspark.ml.pipeline import PipelineModel

# pipeline упрощает процесс внедрения ваших моделей в нужное окружение.

# Перед тем, как модель обучить, мы уже выполнили какие-то преобразования данных.
# Когда мы модель внедряем, нам нужно внедрять и все преобразования над данными.
# Соответственно, появляется место для ошибок (человеческий фактор). Какую-то предобработку могут забыть сделать или сделать неправильно.
# Пайплайны позволяют собрать вашу модель вместе с трансформациями.

In [286]:
redwine_df = spark.read.csv('winequality-red.csv', header=True, inferSchema=True)

# Разделение колонки на отдельные столбцы
split_cols = redwine_df['"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"']

split_cols = split(split_cols, ';')


# Добавление новых столбцов к DataFrame
new_columns = [
    ("fixed_acidity", split_cols.getItem(0)),
    ("volatile_acidity", split_cols.getItem(1)),
    ("citric_acid", split_cols.getItem(2)),
    ("residual_sugar", split_cols.getItem(3)),
    ("chlorides", split_cols.getItem(4)),
    ("free_sulfur_dioxide", split_cols.getItem(5)),
    ("total_sulfur_dioxide", split_cols.getItem(6)),
    ("density", split_cols.getItem(7)),
    ("pH", split_cols.getItem(8)),
    ("sulphates", split_cols.getItem(9)),
    ("alcohol", split_cols.getItem(10)),
    ("quality", split_cols.getItem(11))
]

# Добавление новых столбцов к DataFrame
for column_name, column_expr in new_columns:
    redwine_df = redwine_df.withColumn(column_name, column_expr)
# Удаление исходной колонки
redwine_df = redwine_df.drop('"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"')
for column_name in redwine_df.columns:
    redwine_df = redwine_df.withColumn(column_name, col(column_name).cast('double'))

In [287]:
redwine_df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    5.0|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [328]:
redwine_df = redwine_df.withColumn("target", when(redwine_df["quality"] >= 6, 1).otherwise(0))

In [329]:
features = [col for col in redwine_df.columns if col != 'quality' and col != "target"]
feature = VectorAssembler(inputCols=features, outputCol="features")




In [330]:
rf_classifier = RandomForestClassifier(labelCol="target", featuresCol="features")
feature_vector= feature.transform(redwine_df)
feature_vector.show(10)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+--------------------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|target|            features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+--------------------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|     0|[7.4,0.7,0.0,1.9,...|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|     0|[7.8,0.88,0.0,2.6...|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|

In [331]:
train, test = redwine_df.randomSplit([0.8, 0.2])

pipeline = Pipeline(stages=[feature, rf_classifier])
# Определились с моделью созддаём Pipeline. Указываем массив из трансформаций, которые необходимо выполнить.
# Собранный Pipeline является Estimator'ом, вызываем метод fit, из датафрейма (начального) в одно действие получается готовая модель.

In [332]:
p_model = pipeline.fit(train)

In [333]:
type(p_model)

pyspark.ml.pipeline.PipelineModel

In [334]:
p_model.write().overwrite().save('p_model')
# сохраним модель с перезаписью.

In [335]:
model = PipelineModel.load('p_model')
# Загрузим модель.

In [336]:
prediction = p_model.transform(test)
# применим модель, вызовем метод transrorm, применим его к выборке test

In [337]:
test.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|target|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------+
|          5.2|            0.34|        0.0|           1.8|     0.05|               27.0|                63.0| 0.9916|3.68|     0.79|   14.0|    6.0|     1|
|          5.3|           0.715|       0.19|           1.5|    0.161|                7.0|                62.0|0.99395|3.62|     0.61|   11.0|    5.0|     0|
|          5.6|            0.31|       0.37|           1.4|    0.074|               12.0|                96.0| 0.9954|3.32|     0.58|    9.2|    5.0|     0|
|          5.6|            0.85|       0.05|           1.4

In [338]:
prediction.select([col for col in redwine_df.columns if col != 'quality' and col != "target"]).show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|          5.2|            0.34|        0.0|           1.8|     0.05|               27.0|                63.0| 0.9916|3.68|     0.79|   14.0|
|          5.3|           0.715|       0.19|           1.5|    0.161|                7.0|                62.0|0.99395|3.62|     0.61|   11.0|
|          5.6|            0.31|       0.37|           1.4|    0.074|               12.0|                96.0| 0.9954|3.32|     0.58|    9.2|
|          5.6|            0.85|       0.05|           1.4|    0.045|               12.0|                88.0| 0.9924|3.56|     0.82|   12.9|
|     

In [339]:
evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")

In [340]:
p_accuracy = evaluator.evaluate(prediction)
print("Pipeline model [Accuracy] = %g"% (p_accuracy))
print("Pipeline model [Error] = %g " % (1.0 - p_accuracy))

Pipeline model [Accuracy] = 0.745283
Pipeline model [Error] = 0.254717 


Сделайте самостоятельно Pipeline для Градиентного Бустинга, проверьте точность на тестовой выборке.

In [313]:
gb_classifier = GBTClassifier(labelCol="Survived", featuresCol="features")

pipeline = Pipeline(stages=[indexer_sex, indexer_embarked, feature, gb_classifier])

p_model = pipeline.fit(train)

prediction = p_model.transform(test)

p_accuracy = evaluator.evaluate(prediction)
print("Pipeline model [Accuracy] = %g"% (p_accuracy))
print("Pipeline model [Error] = %g " % (1.0 - p_accuracy))

NameError: name 'indexer_sex' is not defined